#### Stream Reddit data into Google Cloud Platform

In [ ]:
# import dependent libraries
import datetime
import praw
import pandas as pd
import json
import os
from google.cloud import storage
import time

with open('config.text') as json_file:  
    data = json.load(json_file)

# reddit API connection
reddit = praw.Reddit(client_id=data['client_id'],
                     client_secret=data['client_secret'],
                     user_agent=data['user_agent'],
                     username=data['username'],
                     password=data['password'])

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

#### Obtain a Subreddit

To obtain a Subreddit instance, pass the subreddit’s name when calling subreddit on your Reddit instance.

In [ ]:
# assume you have a Reddit instance bound to variable `reddit`
subreddit = reddit.subreddit('galaxys10')

print(subreddit.display_name)
print(subreddit.title) 

In [ ]:
topics_dict = {'submission_id':[],
               'user_id':[],
               'submission_timestamp':[],
               'title':[],
               'body':[]}
count = 0
for submission in subreddit.stream.submissions():
    topics_dict['submission_id'].append(submission.id)
    topics_dict['user_id'].append(submission.author)
    topics_dict['submission_timestamp'].append(submission.created_utc)
    topics_dict['title'].append(submission.title.encode('utf-8').strip())
    topics_dict['body'].append(submission.selftext.encode('utf-8').strip())
    print 'post detected'
    print 'current counter: ' + str(count)
    if count == 50:
        pd.DataFrame(topics_dict).to_csv('./temp_blob.csv', index=False, doublequote=True, sep='|')
        upload_blob('raw_dataset', 'temp_blob.csv', 
                    'reddit/raw_posts/reddit_galaxys10_post_stream_' + str(int(time.time())) + '.csv')
        count = 0

    count += 1